## Installing PyTorch and Pillow == 0.4

In [ ]:
from os import path
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

#accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
!nvidia-smi

In [ ]:
%conda install Pillow

## Imports
All the imports are defined here


In [2]:
%matplotlib inline
import os
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [67]:
!git clone https://github.com/MED-YAHYAOUI/AxisBank_AI_Challenge.git

In [68]:
!ls './AxisBank_AI_Challenge/Forged_Signature_Verification/processed_dataset'

## Helper functions
Set of helper functions



In [5]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

## Configuration Class
A simple class to manage configuration


In [69]:
class Config():
    training_dir = "./AxisBank_AI_Challenge/Forged_Signature_Verification/processed_dataset/training/"
    testing_dir = "./AxisBank_AI_Challenge/Forged_Signature_Verification/processed_dataset/testing/"
    train_batch_size = 64
    train_number_epochs = 500

#### Class Mapping

In [70]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx
  
find_classes(Config.training_dir)[1]

## Custom Dataset Class
This dataset generates a pair of images. 0 for geniune pair and 1 for imposter pair

In [8]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice([item for item in self.imageFolderDataset.imgs if item[1] < 29]) #Considering only genuine images for perfect pair
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1)
        if should_get_same_class:
          while True:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)
            if img0_tuple[1]==img1_tuple[1]:
              break

        else:
          img1_tuple = random.choice([item for item in self.imageFolderDataset.imgs if item[1] == (img0_tuple[1] + 29)]) # Considering a pair of fake + genuine of same person's signature

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

## Using Image Folder Dataset

In [72]:
folder_dataset = dset.ImageFolder(root=Config.training_dir)

In [71]:
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

## Visualising some of the data
The top row and the bottom row of any column is one pair. The 0s and 1s correspond to the column of the image. 1 indiciates dissimilar, and 0 indicates similar.

In [73]:
vis_dataloader = DataLoader(siamese_dataset,
                        shuffle=True,
                        num_workers=16,
                        batch_size=8)
dataiter = iter(vis_dataloader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy())

## Neural Net Definition
We will use a standard convolutional neural network

Its architecture might look like below,

![](https://res.mdpi.com/symmetry/symmetry-10-00385/article_deploy/html/images/symmetry-10-00385-g001.png)

In [12]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(1, 128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(64, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),

            nn.ReflectionPad2d(1),
            nn.Conv2d(32, 16, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(16),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(16, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),


        )

        self.fc1 = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

## Contrastive Loss

In [13]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

## Training Time!

In [14]:
train_dataloader = DataLoader(siamese_dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=Config.train_batch_size)

In [15]:
net = SiameseNetwork().cuda()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(),lr = 0.001)

In [16]:
counter = []
loss_history = [] 
iteration_number= 0

In [ ]:
for epoch in range(0,Config.train_number_epochs):
    for i, data in enumerate(train_dataloader,0):
        img0, img1 , label = data
        img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
        optimizer.zero_grad()
        output1,output2 = net(img0,img1)
        loss_contrastive = criterion(output1,output2,label)
        loss_contrastive.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())
show_plot(counter,loss_history)

In [23]:
# torch.save(net,"/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/saved_model.pt")
# net = torch.load("/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/saved_model.pt")

# filepath = "/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/final_backup_saved_model.pt" # 0.76 threshold

filepath = "./modified_data_loader_saved_model.pt"

torch.save(net.state_dict(), filepath)

#Later to restore:
net.load_state_dict(torch.load(filepath))

In [18]:
net.load_state_dict(torch.load("../input/model-siamese/modified_data_loader_saved_model.pt"))

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/'

### Dataset loader class for inference

In [20]:
class InferenceSiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [143]:
output1,output2,euclidean_distance =predict_img_path("../input/sign-data/Dataset_Signature_Final/Dataset/dataset1/real/00100001.png","../input/sign-data/Dataset_Signature_Final/Dataset/dataset1/real/00100001.png")

In [140]:
euclidean_distance

In [141]:
def predict_img_path(path1,path2):
    img0=path1
    img1=path2
    img0 = Image.open(img0)
    img1 = Image.open(img1)
    img0 = img0.convert("L")
    img1 = img1.convert("L")
    img0_t = tr(img0)
    img1_t = tr(img1)
    img0_t=(img0_t.unsqueeze(0))
    img1_t=(img1_t.unsqueeze(0))
    output1,output2 = net(Variable(img0_t).cuda(),Variable(img1_t).cuda())
    euclidean_distance = F.pairwise_distance(output1, output2)
    imshow(torchvision.utils.make_grid(torch.cat((img0_t,img1_t),0)),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))
    return output1,output2,euclidean_distance

In [127]:
output1

In [123]:
euclidean_distance = F.pairwise_distance(output1, output2)

In [125]:
imshow(torchvision.utils.make_grid(torch.cat((img0_t,img1_t),0)),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))

## Some simple testing

Subject 4 was held out from the training, and will be used to test. The Distance between each image pair denotes the degree of similarity the model found between the two images. Less means it found more similar, while higher values indicate it found them to be dissimilar.

A genuine image on the left is held fixed and is compared with genuine, forged, and different signature.

Genuine : 6,10

Forged: 1,5,7,9

Different : 2,3,4,8

In [74]:
folder_dataset_test = dset.ImageFolder(root=Config.testing_dir)
siamese_dataset = InferenceSiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=True)
dataiter = iter(test_dataloader)
x0,_,_ = next(dataiter)

for i in range(10):
    _,x1,label2 = next(dataiter)
    concatenated = torch.cat((x0,x1),0)
    
    output1,output2 = net(Variable(x0).cuda(),Variable(x1).cuda())
    euclidean_distance = F.pairwise_distance(output1, output2)
    imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))


In [22]:
euclidean_distance

**In conclusion, promising results have been obtained by training a relatively smaller network on only sample dataset provided by the organizers and no additional dataset or the extra dataset provided in the phase two had been used.   This proves the versatility of the Siamese Neural Network for the task of one-shot learning with relatively smaller dataset.**